In [243]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras as ks
from skimage import io
import os
import csv

In [244]:
attr = pd.read_csv('list_attr_celeba.csv')

In [245]:
hair = pd.DataFrame(np.where(attr[['Black_Hair','Bald','Blond_Hair','Brown_Hair','Gray_Hair']] > 0, 1, 0),columns=['Black_Hair','Bald','Blond_Hair','Brown_Hair','Gray_Hair'])

In [246]:
hair = hair[(hair.T != 0).any()]

In [247]:
target_images = hair.index

In [282]:
# Importing cv2 to help work with images
import cv2

# Creating a function to format images
def load_images(path):
    img_data = [] # return the image itself
    index = [] # adds an index to reference image
    x = -1
    for pic in os.listdir(path):
        pic_path = os.path.join(path,pic)
        for img in os.listdir(pic_path):
            x += 1
            if x in target_images:
                img_path = os.path.join(pic_path,img)
                image = cv2.imread(img_path)
                image = cv2.resize(image, (64, 64))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                img_data.append(image)
                index.append(x)
    return(np.array(img_data),np.array(index))

In [283]:
# Setting up fill location for our load_images function
train_path = 'images'

In [284]:
(X, img_num) = load_images(train_path)

In [285]:
img_num

array([     0,      1,      5, ..., 202596, 202597, 202598])

In [286]:
from sklearn.cross_validation import train_test_split

In [287]:
X_train, X_test, y_train, y_test = train_test_split(X, hair, test_size=0.33, random_state=42)

In [288]:
X_train.shape

(86416, 64, 64, 3)

In [330]:
# Importing Modeling Tools
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

In [331]:
# Init Model
model = Sequential()

In [332]:
model.add(Conv2D(16, (3, 3), input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3 )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))

model.add(Activation('softmax'))

In [333]:
# Compiling Model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [334]:
# Training Model
# higher batch and loss 32, .5
model.fit(X_train,y_train, epochs=11, batch_size = 32,
                    validation_data=(X_test,y_test))

Train on 86416 samples, validate on 42564 samples
Epoch 1/11
86416/86416 [==============================] - 1927s 22ms/step - loss: 0.5206 - acc: 0.8244 - val_loss: 0.4287 - val_acc: 0.8518
Epoch 2/11
86416/86416 [==============================] - 1923s 22ms/step - loss: 0.4126 - acc: 0.8655 - val_loss: 0.4238 - val_acc: 0.8816
Epoch 3/11
86416/86416 [==============================] - 1919s 22ms/step - loss: 0.3829 - acc: 0.8767 - val_loss: 0.5292 - val_acc: 0.8385
Epoch 4/11
86416/86416 [==============================] - 1922s 22ms/step - loss: 0.3658 - acc: 0.8832 - val_loss: 0.6059 - val_acc: 0.8553
Epoch 5/11
86416/86416 [==============================] - 1920s 22ms/step - loss: 0.3521 - acc: 0.8883 - val_loss: 0.3787 - val_acc: 0.8708
Epoch 6/11
86416/86416 [==============================] - 1922s 22ms/step - loss: 0.3416 - acc: 0.8921 - val_loss: 0.4965 - val_acc: 0.8659
Epoch 7/11
86416/86416 [==============================] - 1917s 22ms/step - loss: 0.3312 - acc: 0.8964 - val_l

In [335]:
from sklearn.metrics import classification_report

In [342]:
# Saving predictions for new model
predictions = model.predict_classes(X_test)

In [346]:
print(y_test.shape)
predictions.shape

(42564, 5)


(42564,)

In [376]:
from keras.utils import np_utils
pred = np_utils.to_categorical(predictions)

In [377]:
pred.shape

(42564, 5)

In [387]:
# Printing metrics
print(classification_report(pred,y_test,target_names=['Black Hair','Bald','Blond Hair','Brown Hair','Gray Hair']))

             precision    recall  f1-score   support

 Black Hair       0.97      0.88      0.92     17675
       Bald       0.77      0.81      0.79      1403
 Blond Hair       0.92      0.90      0.91     10216
 Brown Hair       0.75      0.94      0.83     10897
  Gray Hair       0.74      0.86      0.79      2373

avg / total       0.88      0.90      0.89     42564

